Create a thiophene structure and run Gaussian and LAMMPS

Import some standard python modules

In [2]:
import os,sys

This will use the resource creating in the resource.ipynb exmaple. This can be either a local  or remote  as long as the specified resource has 

In [5]:
from streamm import resource

In [7]:
res_tag = 'local'  # Change this to remote to run the calculations remotely 
res_i = resource.Resource(res_tag)
res_i.dump_json()

In [4]:
import buildingblock

Build structure (this could be read in from a file)

In [5]:
bbTh = buildingblock.Container('thiophene')
symbols = ['C','C','C','C','S','H','H','H','H']
positions = [ ]
positions.append([-1.55498576,-1.91131218,-0.00081000])
positions.append([-0.17775976,-1.91131218,-0.00081000])
positions.append([0.34761524,-0.57904218,-0.00081000])
positions.append([-0.65884476,0.36101082,0.00000000])
positions.append([-2.16948076,-0.35614618,-0.00000800])
positions.append([-2.18966076,-2.79526518,-0.00132100])
positions.append([0.45389024,-2.80145418,-0.00106400])
positions.append([1.41682424,-0.35961818,-0.00138200])
positions.append([-0.51943676,1.44024682,0.00064700])
for i in range(len(symbols)):
    pt_i = buildingblock.BBatom(symbols[i])
    pos_i = positions[i]
    bbTh.add_partpos(pt_i,pos_i)

Write an xyz file that can be viewed in viewer, such as VMD or Avogadro

In [6]:
bbTh.write_xyz()

Create a gaussian calculation 

In [7]:
import gaussian

In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
gaus_i = gaussian.Gaussian('th_gaus')

Add thiophene structure to calculation

In [10]:
gaus_i.set_strucC(bbTh)

Set the resource. This gives the calculation information on the location of files and if files need to coppied to a remote computer

In [11]:
gaus_i.set_resource(res_i)

Copy templates to scratch

In [12]:
file_type = 'templates'
file_key = 'run'
file_name = "gaussian.sh"
from_dirkey = 'templates'
to_dirkey = 'scratch'
gaus_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  templates run gaussian.sh templates scratch
copying /Users/tkemper/Development/tools/examples/../templates/gaussian.sh to /Users/tkemper/Development/tools/examples/scratch/th_gaus/gaussian.sh 


In [13]:
file_type = 'templates'
file_key = 'com'
file_name = "gaussian.com"
from_dirkey = 'templates'
to_dirkey = 'scratch'
gaus_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  templates com gaussian.com templates scratch
copying /Users/tkemper/Development/tools/examples/../templates/gaussian.com to /Users/tkemper/Development/tools/examples/scratch/th_gaus/gaussian.com 


In [14]:
print gaus_i.files[file_type]

{'com': 'gaussian.com', 'run': 'gaussian.sh'}


Change to scratch directory where files are 

In [15]:
os.chdir(gaus_i.dir['scratch'])

Load templates as strings

In [16]:
gaus_i.load_str('templates','com')

In [17]:
gaus_i.load_str('templates','run')

In [18]:
print gaus_i.str

{'run': '\ng09 < <input_com> >&  <tag>.log\n\nformchk <tag>.chk\n \n\n\n\n', 'com': '%chk=<tag>.chk\n#P <commands> \n\n<tag> \n\n<charge> <spin_mult>\n<coord>\n\n\n'}


Set calculation properties specified in .com template

In [19]:
gaus_i.properties['commands'] = 'HF/3-21G SP'
gaus_i.properties['charge'] = 0
gaus_i.properties['spin_mult'] = 1
gaus_i.properties['coord'] = gaus_i.strucC.write_coord()

Write .com file and record it in calculation's input files

In [20]:
gaus_i.replacewrite_prop('com','input','com','%s.com'%(gaus_i.tag))

Set calculation properties specified in .sh template

In [21]:
gaus_i.properties['input_com'] = gaus_i.files['input']['com']

Write run script for running calculation

In [22]:
gaus_i.replacewrite_prop('run','scripts','run','%s.sh'%(gaus_i.tag))

Add log file to output files

In [23]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(gaus_i.tag)
gaus_i.add_file(file_type,file_key,file_name)

In [24]:
file_type = 'output'
file_key = 'fchk'
file_name = "%s.fchk"%(gaus_i.tag)
gaus_i.add_file(file_type,file_key,file_name)

Checkpoint calculation by outputing json file

In [25]:
os.chdir(gaus_i.dir['home'])
gaus_i.dump_json()
os.chdir(gaus_i.dir['scratch'])

Run calculation

In [ ]:
gaus_i.run()

Check the status of the calculation

In [26]:
gaus_i.check()
print gaus_i.meta['status']

Running check in local directory /Users/tkemper/Development/tools/examples/scratch/th_gaus 
finished


Analyize output of calculation

In [27]:
gaus_i.analysis()

Set charges of structure to mulliken charges from the calculation

In [28]:
gaus_i.strucC.properties['charge'] = 0.0 
for p,pt in gaus_i.strucC.particles.iteritems(): # properties["charge"] 
    pt.properties["charge"] = round(pt.properties["mulliken"],3)
    gaus_i.strucC.properties['charge']  += pt.properties["charge"]
    print pt.properties["charge"]

-0.71
-0.2
-0.2
-0.71
0.766
0.28
0.247
0.247
0.28


Check that rounding has not affected the total charge

In [29]:
print "Total charge %f "%(gaus_i.strucC.properties['charge'])

Total charge 0.000000 


Store output 

In [30]:
gaus_i.store()

runnning store function in /Users/tkemper/Development/tools/examples/scratch/th_gaus 
Adding input file th_gaus.com with key com 
> in cp_file  input compressed th_gaus_input.tgz scratch storage
copying /Users/tkemper/Development/tools/examples/scratch/th_gaus/th_gaus_input.tgz to /Users/tkemper/Development/tools/examples/storage/th_gaus/th_gaus_input.tgz 
Adding scripts file th_gaus.sh with key run 
> in cp_file  scripts compressed th_gaus_scripts.tgz scratch storage
copying /Users/tkemper/Development/tools/examples/scratch/th_gaus/th_gaus_scripts.tgz to /Users/tkemper/Development/tools/examples/storage/th_gaus/th_gaus_scripts.tgz 
Adding output file th_gaus.log with key log 
Adding output file th_gaus.fchk with key fchk 
> in cp_file  output compressed th_gaus_output.tgz scratch storage
copying /Users/tkemper/Development/tools/examples/scratch/th_gaus/th_gaus_output.tgz to /Users/tkemper/Development/tools/examples/storage/th_gaus/th_gaus_output.tgz 
No files of type data present


Check that that data can be pulled into the local scratch directory 

In [31]:
gaus_i.pull()

runnning pull function in /Users/tkemper/Development/tools/examples/scratch/th_gaus 
> in cp_file  output compressed th_gaus_output.tgz storage scratch
copying /Users/tkemper/Development/tools/examples/storage/th_gaus/th_gaus_output.tgz to /Users/tkemper/Development/tools/examples/scratch/th_gaus/th_gaus_output.tgz 


Guassian calculation of thiophene is finished, return to home directory of calculation

In [32]:
os.chdir(gaus_i.dir['home'])

In [33]:
gaus_i.dump_json()

Start lammps calculation

In [34]:
import lammps

In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
lmp_i = lammps.LAMMPS('th_lmp')

In [37]:
lmp_i.set_resource(res_i)


Convert the structure from the gaussian output to the lammmps calculation

In [38]:
lmp_i.convert_gaussian(gaus_i)

Set box size

In [39]:
lmp_i.strucC.lat_cubic(25.0)

Find nieghbors bassed on distances and covalent radii

In [40]:
lmp_i.strucC.bonded_nblist.guess_nblist(lmp_i.strucC.lat,lmp_i.strucC.particles,lmp_i.strucC.positions,"cov_radii",radii_buffer=1.25)

Read in force-field parameters

In [41]:
lmp_i.read_param("conj.param")

Guess force-field atom types 

In [42]:
lmp_i.strucC.guess_oplsa()

Set bonded interactions

In [43]:
lmp_i.strucC.bonded_bonds()
lmp_i.strucC.bonded_angles()
lmp_i.strucC.bonded_dih()

Clean up force-field parameters so only the parameters for the interactions in the molecule are included

In [44]:
lmp_i.set_ffparam()

Read in templates

In [45]:
file_type = 'templates'
file_key = 'in'
file_name = "lammps_sp.in"
from_dirkey = 'templates'
to_dirkey = 'scratch'
lmp_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  templates in lammps_sp.in templates scratch
copying /Users/tkemper/Development/tools/examples/../templates/lammps_sp.in to /Users/tkemper/Development/tools/examples/scratch/th_lmp/lammps_sp.in 


In [46]:
file_type = 'templates'
file_key = 'run'
file_name = "lammps.sh"
from_dirkey = 'templates'
to_dirkey = 'scratch'
lmp_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  templates run lammps.sh templates scratch
copying /Users/tkemper/Development/tools/examples/../templates/lammps.sh to /Users/tkemper/Development/tools/examples/scratch/th_lmp/lammps.sh 


In [47]:
print lmp_i.files[file_type]

{'run': 'lammps.sh', 'in': 'lammps_sp.in'}


Write input files in scratch directory

In [48]:
os.chdir(lmp_i.dir['scratch'])

In [49]:
lmp_i.load_str('templates','run')

In [50]:
lmp_i.load_str('templates','in')

In [51]:
lmp_i.write_data()

In [52]:
lmp_i.replacewrite_prop('in','input','in','%s.in'%(lmp_i.tag))

In [53]:
lmp_i.properties['input_in'] = lmp_i.files['input']['in']

In [54]:
lmp_i.replacewrite_prop('run','scripts','run','%s.sh'%(lmp_i.tag))

Add output log file to output files

In [55]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(lmp_i.tag)
lmp_i.add_file(file_type,file_key,file_name)

In [56]:
os.chdir(lmp_i.dir['home'])
lmp_i.dump_json()
os.chdir(lmp_i.dir['scratch'])

Run calculation

In [ ]:
lmp_i.run()

In [57]:
lmp_i.check()
print lmp_i.meta['status']

Running check in local directory /Users/tkemper/Development/tools/examples/scratch/th_lmp 
finished


In [93]:
lmp_i.analysis()
print lmp_i.meta['status']

finished


In [99]:
print lmp_i.properties['run_cnt']
run_i = lmp_i.properties['run_list'][0]
print run_i.properties['timestep'],run_i.timeseries.keys()
print run_i.timeseries['step']
print run_i.timeseries['toteng']
print run_i.timeseries['poteng']

1
0.5 ['e_tail', 'e_dihed', 'temp', 'e_long', 'e_bond', 'e_coul', 'volume', 'step', 'e_impro', 'e_vdwl', 'e_angle', 'press', 'toteng', 'lz', 'poteng', 'lx', 'ly']
[0.0, 1.0]
[30.77175, 30.77175]
[30.77175, 30.77175]


In [ ]:
import gromacs

In [ ]:
gro_i = gromacs.GROMACS("th")

In [ ]:
gro_i.read_itp('conj.itp')

In [ ]:
lmp_i.convert_GROMACS(gro_i)

In [ ]:
lmp_i.write_data()

In [ ]:
lmp_i.write_param("t1.param")

In [ ]:
del lmp_i